In [185]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from keras.preprocessing import image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import PIL
import io
import html
import time
import tensorflow

In [186]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [325]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 1920, 1080);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'Click here or on the video to stop</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 1920; //video.videoWidth;
      captureCanvas.height = 1080; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [326]:
import numpy as np
import cv2
#green
def hs1(path):
  import cv2
  import numpy as np

  ## Read
  img = cv2.imread(path)
  
  ## convert to hsv
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  ## mask of green (36,25,25) ~ (86, 255,255)
  mask = cv2.inRange(hsv, np.array([25, 75, 100]), np.array([92, 255, 255]))
  #mask = cv2.inRange(hsv, np.array([94, 80, 2]), np.array([126, 255, 255]))

  ## slice the green
  imask = mask>0
  green = np.zeros_like(img, np.uint8)
  green[imask] = img[imask]

  ## save 
  cv2.imwrite("green.png", green)

import numpy as np
import cv2
boundaries = [
    ([0, 120, 0], [140, 255, 100]),
    ([25, 0, 75], [180, 38, 255])
]

#pink
def hs(frame):
    #frame = cv2.imread(frame)
    lower, upper = boundaries[0]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask1 = cv2.inRange(frame, lower, upper)

    lower, upper = boundaries[1]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask2 = cv2.inRange(frame, lower, upper)

    # for i,(lower, upper) in enumerate(boundaries):
    # 	# create NumPy arrays from the boundaries
    # 	lower = np.array(lower, dtype = "uint8")
    # 	upper = np.array(upper, dtype = "uint8")

    # 	# find the colors within the specified boundaries and apply
    # 	# the mask
    # 	if(i==0):
    # 		print "Harish"
    # 		mask1 = cv2.inRange(frame, lower, upper)
    # 	else:
    # 		print "Aadi"
    # 		mask2 = cv2.inRange(frame, lower, upper)
    mask = cv2.bitwise_or(mask1, mask2)
    output = cv2.bitwise_and(frame, frame, mask=mask)
    # show the images
    # cv2.imshow("images", mask)
    # cv2.imshow("images", output)
    cv2.imwrite("green.png", output)
    return output

In [315]:
classes = {1:'Colors', 0:'Women', 5:'Learn', 6:'Call', 8:'Drawer'}

In [242]:
model = tensorflow.compat.v1.keras.experimental.load_from_saved_model("/content/drive/MyDrive/2.30")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/saving/saved_model_experimental.py:412: UserWarning: `tf.keras.experimental.load_from_saved_model` is deprecatedand will be removed in a future version. Please switch to `tf.keras.models.load_model`.
  warnings.warn('`tf.keras.experimental.load_from_saved_model` is deprecated'


In [334]:
from keras.preprocessing import image
import cv2
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
label_classes = ''
#label_map = classes#{j:i for i, j in classes.items()}
while True:
    """#videoload
    name = '/content/drive/MyDrive/lsa64_cut/all_cut/014_008_002.mp4'
    cap = cv2.VideoCapture(name)
    ret, frame = cap.read()  # extract frame
    if ret is False:
        break
    frame = hs(frame)"""

    #webcam
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    img = PIL.Image.fromarray(js_to_image(js_reply["img"]))
    img.save('green.png')
    hs1('green.png')
    frame = cv2.imread('green.png')

    
    imge = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    print(imge.shape)
    cv2.imwrite('/content/drive/MyDrive/gesture.png', imge)

    imgx = image.img_to_array(image.load_img('/content/drive/MyDrive/gesture.png', target_size=(28, 28)))
    img = tensorflow.image.rgb_to_grayscale(imgx)
    img = np.array(img / 255.)
    x = np.expand_dims(img, axis=0)

    images = np.vstack([x])
    y = model.predict(images)
    if max(y[0]) > 0.75:
      y_classes = np.argmax(y, axis=-1)[0]
      print(y_classes)
      if y_classes in classes:
        #print(y)
        plt.axis('off')
        plt.imshow(imge)
        plt.show()
        print(classes[y_classes])
        label_classes += " " + classes[y_classes]
        if len(label_classes) > 24:
          label_classes = classes[y_classes]
    # create transparent overlay for bounding box
    bbox_array = np.zeros([1920,1080,4], dtype=np.uint8)


    # get face bounding box for overlay
    
    bbox_array = cv2.putText(bbox_array,label_classes,(15, 275),cv2.FONT_HERSHEY_SIMPLEX , 0.75,(255, 255, 255),2,cv2.LINE_AA) 

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
9
(1080, 1920)
(1080, 1920)
(1080, 1920)
9
